# Week of 2025.3.10

Last week I tried to get the slip PR in a reviewable and mergable state. One of the points that Eric made was that I should make sure to update the get_jacobian function with my new state variable.

> Related to the above, the full jacobian should be a bit larger when adding another dependent variable (slip). I doubt the jacobian function would modified by Fred to > account for this. In fact, I'm not sure if he broke the ability to do operator splitting and implicit time stepping when he added slip as a new variable. The truly correct thing would be to make sure the jacobian is correct, but that could be a lot of work, especially in the super general framework where Fred allows for nonlocal dependence on fields in the term that is treated implicitly. I'm not sure that it's worth your time to figure this out and implement the needed changes. But at least we ought to include some kind of comment in the code and/or repo about this. Personally, I would recommend that you not update this full jacobian, but instead make note of it. Then as part of the new set of code changes, add support for an implicit term that is local in space. That set of changes will include the correct treatment of slip-dependence of the implicit term.

After a bit of investigation at the end of last week, I found that in order to handle one of the source terms implicity it was nessesary to mark a specific source as `implicit` and also change the timestepping method to [stang](https://en.wikipedia.org/wiki/Strang_splitting), which is basically a way to solve subproblems of a differential equation seperately and still guarantee second order accuracy. 

Before we merge the slip code, we want to make sure the general jacobian is correct. 

However, the more important goal is to implement "local" implicit updates rather.


## 1.0 Imports

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Modify base path for depending on your file structure.
BASE_PATH = "/Users/paxton/git"

# Specify path where .pkl files are located
target_dir = f"{BASE_PATH}/quail_volcano/scenarios/simple_1D_test"
# Specify path for Quail source code
source_dir = f"{BASE_PATH}/quail_volcano/src"
# Change to working directory
os.chdir(target_dir)


# Import quail modules
os.chdir(source_dir)

import meshing.tools as mesh_tools

import numerics.helpers.helpers as helpers
import numerics.timestepping.tools as stepper_tools

import physics.zerodimensional.zerodimensional as zerod
import physics.euler.euler as euler
import physics.navierstokes.navierstokes as navierstokes
import physics.scalar.scalar as scalar
import physics.chemistry.chemistry as chemistry
import physics.multiphasevpT.multiphasevpT as multiphasevpT

import processing.readwritedatafiles as readwritedatafiles
import processing.post as post
import processing.plot as plot
import processing.animate as animate

import solver.DG as DG
import solver.ADERDG as ADERDG
import solver.tools as solver_tools

import time
from IPython.display import HTML
import multiprocessing as mp  
from multidomain import Domain, Observer

os.chdir(target_dir)

## 1.1 Getting the slip code to a place where it is merge ready

- When the default viscosity is set, set all partial derivatives with respect to viscosity equal to zero. 
- The following example is solved with implicit time stepping and appears to be reasonable. 

In [7]:
ani = animate.animate_conduit_pressure("slip_variable_addition", iterations=100, file_prefix="short_plug_implicit_source")

HTML(ani.to_html5_video())

/Users/paxton/git/quail_volcano/scenarios/simple_1D_test


## 2.1 Adding a jacobian for conduit wall drag 

In the future, in order to support implicit timestepping for the conduit wall friction source term I will want to add a `get_jacobian` for that source. The derivate of momentum with respect to slip aught to look something like: 

$$
\begin{align}
\frac{d_{mom}}{d slip} =& \frac{d \tau}{d slip} \frac{d mom}{d \tau} \\

\tau(slip) =& \tau_r - (\tau_r - \tau_p) e^{-slip/D_c} \\

momentum =& \frac{d \rho u}{d t} = - \frac{-2 \tau}{R} \\

\frac{d \tau}{d slip} =& - \frac{1}{D_c} \tau \\

\frac{d_{mom}}{d \tau} =& - \frac{2}{R} \\

\frac{d_{mom}}{d slip} =& \frac{1}{D_c} \frac{2}{R} \tau 
\end{align}
$$

However, to just merge this initial slip functionality that should not be nesessary. 

### 3.1 Verify that I did not break anything really obvious with my changes

In [4]:
ani = animate.animate_conduit_pressure("slip_variable_addition", iterations=100, file_prefix="short_plug_dynamic")

HTML(ani.to_html5_video())

/Users/paxton/git/quail_volcano/scenarios/simple_1D_test


## 4.1 Review of timesteps and integrators

I found some [nice review videos](https://www.youtube.com/watch?v=HOWJp8NV5xU&list=PLMrJAkhIeNNTYaOnVI3QpH7jgULnAmvPA&index=44) from Steve Brunton which review FE, BE, error analysis, and Runge-Kutta integrators. 

Some relevant terms:
- stiff: unstable except for very small time steps. 
- explicit numerical method: Calculates $X_{K+1}$ as a function of $X_k$, for instance forward euler: $X_{k+1} = X_k + \Delta t f(X_k)$. 
- implicit numerical method: Calculates $K_{k+1}$ as a function of $X_{k+1}$, for instance backward euler: $X_{K+1} = X_k + \Delta t f(X_{k+1})$


Generally, implicit methods are computationally more complex but have much better stability conditions. 

